In [1]:
from utils.load_utils import get_exp_meta

# EXPERIMENT META 
exp_code = "b1382"
exp_meta = get_exp_meta(exp_code)
run_code = "10"
desired_sampling_freq = 50

In [2]:
import os

# JUPYTER
# %matplotlib widget
%reload_ext autoreload
%autoreload 2

# FILENAMES
working_dir = os.getcwd()
data_dir = working_dir + "/data"
results_dir = working_dir + "/results/" + run_code
datapath = data_dir + "/" + exp_code + ".txt"
picklepath = data_dir + '/' + exp_code + "_" + str(desired_sampling_freq) + "hz.pickle"

# MAKE RESULTS DIR IF NOT EXISTS
if os.path.exists(results_dir):
    if os.listdir(results_dir):
        print("WARNING: results dir already contains data")
else:
    os.makedirs(results_dir)

## Import data

In [ ]:
from utils.load_utils import import_brava_data
import matplotlib.pyplot as plt

# IMPORT
args = dict((k, exp_meta[k]) for k in ["t_start", "t_end", "ROIs", "LVDT_readjustments", "LVDT_method"] if k in exp_meta)
data = import_brava_data(datapath, picklepath, downsample_factor=(exp_meta["sampling_freq"] / desired_sampling_freq), **args)

# HAVE A LOOK
print("\n", data.head())
# plt.close("all")
plt.plot(data["TIME"], data["SHEAR STRESS"])
# plt.show()

# TO CSV FOR JULIA
csvtspath = data_dir + '/' + exp_code + "_" + str(desired_sampling_freq) + "hz_ts.csv"
if not os.path.isfile(csvtspath):
    data.to_csv(csvtspath, columns=["SHEAR STRESS"])

## Generate windows

In [ ]:
# sliding windows
win_length = 6000
win_offset = 3000

num_wins = (len(data) - win_length) // win_offset
windows = []
for i in range(num_wins):
    windows.append((i*win_offset, i*win_offset + win_length))
print("Number of windows:", num_wins)

In [15]:
# turn ROIs into windows
# windows = []
# for roi in exp_meta["ROIs"]:
#     masked = data[ data["TIME"].between(roi[0], roi[1]) ]
#     windows.append((masked.index[0], masked.index[-1]))

## Set dynamics parameters

In [5]:
import numpy as np
dynsys_params = {

    # CALCULATE BEST m, tau BY CAO (1997)
    "tau_to_try": np.array([2,5,8,15,23,32,45,60,70,85,100]),
    "m_to_try": np.arange(1,13,1),
    "E1_thresh": 0.9,
    "E2_thresh": 0.9,
    "eps_over_L0": 0.05,
    "n_neighbors": 13,

    # LYAPUNOV CALCULATIONS
    "sampling": ['rand',None], # to calculate LE spectrum

    # KRAEMER DELAY EMBEDDING
    "tau_max_kraemer": 100,
    "valbins": 0.005, # normalised value binning interval for shannon entropy

}

## Package everything up

In [6]:
import pickle
import json
from utils.load_utils import JsonEncoder

# COLLECT OPTS
opts = {
    "exp_code": exp_code,
    "exp_meta": exp_meta,
    "sampling_freq": desired_sampling_freq,
    "run_code": run_code,
    "working_dir": working_dir,
    "data_dir": data_dir,
    "results_dir": results_dir,
    "datapath": datapath,
    "picklepath": picklepath,
    "csvtspath": csvtspath,
    "windows": windows,
    "dynsys_params": dynsys_params
}

# WRITE TO PICKLE
opts_picklepath = results_dir + "/opts.pickle"
with open(opts_picklepath, 'wb') as f:
    pickle.dump(opts, f)

# WRITE TO JSON
opts_jsonpath = results_dir + "/opts.json"
with open(opts_jsonpath, 'w') as f_:
    f_.write( json.dumps(opts, indent=4, cls=JsonEncoder) )
